In [1]:
import requests
import json
from bs4 import BeautifulSoup
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder
from OSMPythonTools.overpass import Overpass

In [2]:
import pandas as pd
import numpy as np
import osmnx as ox
import folium
from shapely.geometry import Point, LineString, Polygon
import geopandas as gpd
import geopy.distance
import networkx as nx
import math
import copy as cp
from sklearn import preprocessing
import shapely.wkt
import csv
from tqdm import tqdm
from math import radians
from numpy import inf
import branca.colormap as cm
import random
from folium.plugins import HeatMap
from shapely.geometry import Polygon
import json
import geojson
from geojson import Feature, FeatureCollection

from functools import partial
from shapely.ops import transform
import pyproj

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import json

In [4]:
import geopy
import geopy.distance
import pandas as pd

In [5]:
# настройка экрана
pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 200)

In [6]:
  # города, по которым собираем информацию
regions = ['Москва',
           'Московская область', 
           'Санкт-Петербург', 
           'Новосибирск', 
           'Екатеринбург', 
           'Казань', 
           'Нижний Новгород', 
           'Красноярск', 
           'Челябинск', 
           'Самара', 
           'Уфа', 
           'Ростов-на-Дону', 
           'Краснодар', 
           'Омск', 
           'Воронеж', 
           'Пермь', 
           'Волгоград']


In [7]:
def parser_objects(location:list, objects:list) -> None:
    '''
    функция собирает данные через API с OSM
    функция на вход принимает два аргумента:
    location: list -> регионы,
    objects: list -> кортеж (тег - объект),
    функция ничего не возвращает,
    но сохраняет данные в формате csv
    '''
    assert type(location) == list
    assert type(objects) == list
    cities_dfs = []
    for i in range(len(location)):
        # вызываем геокодер
        nominatim = Nominatim()
        
        # закидываем наши регионы в метод query, который обрабатывает регионы под нужный формат запроса
        region = nominatim.query(location[i]) 
        city_objects = []
        for j in range(len(objects)):
            print(location[i], objects[j])
            
            # создаем автоматически запрос на языке OverPass QL
            query_obj = overpassQueryBuilder(area=region, # выбираем регион для поиска объектов
                                             elementType='node', # точечный тип объекта
                                             selector=f'{objects[j][0]}={objects[j][1]}') # отбираем объект
            
            
            # подключаемся к API OSM через класс Overpass по ссылке: http://overpass-api.de/api/
            overpass = Overpass()
            
            # закидываем в API наш запрос и получаем выходной объект
            obj = overpass.query(query_obj)
            
            # переводим объект в фрмат json, выбираем 'elements' и закидываем в df
            df = pd.DataFrame(obj.toJSON()['elements'])
            if df.empty:
                continue
            
            # дополнительные параметры можно взять здесь (при необходимости)
            #params = df['tags'].apply(lambda x: pd.Series(x))
            #df = df.join(params['name'])
            
            # отсекаем все прочие столбцы
            df = df[['id', 'lat', 'lon']]
            df['object_type'] = objects[j][0]
            df['object_name'] = objects[j][1]
            df['city'] = location[i]
            city_objects.append(df)
            
            # print(df.tail())
            # print('-' * 40)
            
            # Для сохранения df
            # df.to_csv(f'{location[i]}: {objects[j][0]}-{objects[j][1]}.csv') 
        cities_dfs.append(pd.concat(city_objects))
    return pd.concat(cities_dfs)

# regions = ['Москва','Московская область']

tag_objects = [('highway','bus_stop'), 
               ('amenity', 'hospital'),
               ('amenity', 'clinic'),
               ('subway','yes'),
               ('railway', 'station'),
               ('tourism', 'hotel'),
               ('shop', 'supermarket'),
               ('shop', 'mall'),
               ('shop', 'general'),
               ('shop', 'department_store'),
               ('office', 'yes'),
               ('amenity', 'pharmacy'),
               ('building', 'public'),
               ('amenity', 'university'),
               ('amenity', 'college'),
               ('leisure', 'stadium'),
               ('amenity', 'cafe'),
               ('amenity','restaurant'),
               ('amenity', 'fast_food'),
               ('amenity', 'food_court'),
               ('amenity', 'fuel'),
               ('amenity', 'atm'),
               ('amenity', 'bank'),
               ('amenity', 'bureau_de_change'),
               ('amenity', 'cinema'),
               ('amenity', 'nightclub'),
               ('amenity', 'theatre'),
               ('amenity', 'police'),
               ('amenity', 'courthouse')              
              ]

all_objects = parser_objects(regions, tag_objects)

Москва ('highway', 'bus_stop')
Москва ('amenity', 'hospital')
Москва ('amenity', 'clinic')
Москва ('subway', 'yes')
Москва ('railway', 'station')
Москва ('tourism', 'hotel')
Москва ('shop', 'supermarket')
Москва ('shop', 'mall')
Москва ('shop', 'general')
Москва ('shop', 'department_store')
Москва ('office', 'yes')
Москва ('amenity', 'pharmacy')
Москва ('building', 'public')
Москва ('amenity', 'university')
Москва ('amenity', 'college')
Москва ('leisure', 'stadium')
Москва ('amenity', 'cafe')
Москва ('amenity', 'restaurant')
Москва ('amenity', 'fast_food')
Москва ('amenity', 'food_court')
Москва ('amenity', 'fuel')
Москва ('amenity', 'atm')
Москва ('amenity', 'bank')
Москва ('amenity', 'bureau_de_change')
Москва ('amenity', 'cinema')
Москва ('amenity', 'nightclub')
Москва ('amenity', 'theatre')
Москва ('amenity', 'police')
Москва ('amenity', 'courthouse')
Московская область ('highway', 'bus_stop')
Московская область ('amenity', 'hospital')
Московская область ('amenity', 'clinic')
Моско

Челябинск ('amenity', 'fast_food')
Челябинск ('amenity', 'food_court')
Челябинск ('amenity', 'fuel')
Челябинск ('amenity', 'atm')
Челябинск ('amenity', 'bank')
Челябинск ('amenity', 'bureau_de_change')
Челябинск ('amenity', 'cinema')
Челябинск ('amenity', 'nightclub')
Челябинск ('amenity', 'theatre')
Челябинск ('amenity', 'police')
Челябинск ('amenity', 'courthouse')
Самара ('highway', 'bus_stop')
Самара ('amenity', 'hospital')
Самара ('amenity', 'clinic')
Самара ('subway', 'yes')
Самара ('railway', 'station')
Самара ('tourism', 'hotel')
Самара ('shop', 'supermarket')
Самара ('shop', 'mall')
Самара ('shop', 'general')
Самара ('shop', 'department_store')
Самара ('office', 'yes')
Самара ('amenity', 'pharmacy')
Самара ('building', 'public')
Самара ('amenity', 'university')
Самара ('amenity', 'college')
Самара ('leisure', 'stadium')
Самара ('amenity', 'cafe')
Самара ('amenity', 'restaurant')
Самара ('amenity', 'fast_food')
Самара ('amenity', 'food_court')
Самара ('amenity', 'fuel')
Самара 

Ссылка на теги по магазинам: https://wiki.openstreetmap.org/wiki/RU:Key:shop
Ссылка на теги по зданиям: https://wiki.openstreetmap.org/wiki/RU:Key:building
доп. ссылка: https://wiki.openstreetmap.org/wiki/RU:Key:amenity 

In [8]:
regions

['Москва',
 'Московская область',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [9]:
#city: north, west, N

city_params_dict = {
    'Москва':(56.031959, 36.797755, 343), #55.911274, 37.369340, 140) 
 # 'Московская область':,
     'Санкт-Петербург': (60.246734, 29.424817, 220),
     'Новосибирск': (55.197927, 82.740514, 150),
     'Екатеринбург': (56.982824, 60.383625, 144),
     'Казань': (55.919676, 48.846786, 144),
     'Нижний Новгород': (56.420110, 43.719819, 84),
     'Красноярск': (56.133141, 92.652852, 100),
     'Челябинск': (55.318748, 61.241503, 100),
     'Самара': (53.426067, 50.007146, 127),
     'Уфа': (54.959111, 55.765642, 170),
     'Ростов-на-Дону': (47.366032, 39.527174, 80),
     'Краснодар': (45.140525, 38.843852, 100),
     'Омск': (55.140361, 73.095170, 130),
     'Воронеж': (51.816681, 39.013137, 130),
     'Пермь': (58.180438, 55.797528, 170),
     'Волгоград': (48.894814, 44.069950, 170)
                   }

In [10]:
def findloc(lat, lon, poly):
    """
    Принимает на вход широту и долготу объекта, координатную сетку 
    Возвращает индексы квадрата в сетке с наименьшим геометрическим расстоянием от центра квадрата до заданной точки
    """
    j=np.argmin(np.abs(poly[0,:,0]-lat))
    k=np.argmin(np.abs(poly[1,0,:]-lon))
    return [j,k]

In [11]:
all_objects['compound_obj'] = all_objects['object_type'] + all_objects['object_name']

In [12]:
cell_side = 300
mapped_objects_dfs = []
for curr_city in tqdm(city_params_dict):

    #левый верхний угол
    north, west, N = city_params_dict[curr_city]

    #начало сетки
    start = geopy.Point(north, west)

    #откладываем сетку на восток и юг
    distance_longitude = geopy.distance.distance(kilometers=(N*cell_side/1000))
    distance_latitude = geopy.distance.distance(kilometers=(N*cell_side/1000))

    #считаем расстояние и получаем координаты востока и юга
    destination_east = distance_longitude.destination(point=start, bearing=90)
    destination_south = distance_latitude.destination(point=start, bearing=180)

    east = destination_east.longitude
    south = destination_south.latitude

    sn = (south - north)/N
    ew = (east - west)/N

    upper_left = np.mgrid[north:south:sn, west:east:ew]
    upper_right = np.mgrid[north:south:sn, west+ew:east+ew:ew]
    lower_left=np.mgrid[north+sn:south+sn:sn, west:east:ew]
    lower_right=np.mgrid[north+sn:south+sn:sn, west+ew:east+ew:ew]

    grid = pd.DataFrame()

    grid["upper_left_lat"] = upper_left[0].reshape(N * N)
    grid["upper_left_lon"] = upper_left[1].reshape(N * N)

    grid["upper_right_lat"] = upper_right[0].reshape(N * N)
    grid["upper_right_lon"] = upper_right[1].reshape(N * N)

    grid["lower_left_lat"] = lower_left[0].reshape(N * N)
    grid["lower_left_lon"] = lower_left[1].reshape(N * N)

    grid["lower_right_lat"] = lower_right[0].reshape(N * N)
    grid["lower_right_lon"] = lower_right[1].reshape(N * N)

    grid["X_ID"] = np.mgrid[0:N:1,0:N:1][0].reshape(N * N,)
    grid["Y_ID"] = np.mgrid[0:N:1,0:N:1][1].reshape(N * N,)

    grid["cell_center_lon"] = (grid.upper_right_lon - grid.upper_left_lon)/2 + grid.upper_left_lon
    grid["cell_center_lat"] = (grid.upper_right_lat - grid.lower_right_lat)/2 + grid.lower_right_lat

    cell_center = np.zeros(shape=(2,N, N))

    cell_center[0] = np.array(grid.cell_center_lat).reshape(N, N)
    cell_center[1] = np.array(grid.cell_center_lon).reshape(N, N)
    city_objects = all_objects[all_objects['city'] == curr_city].copy()
    city_objects['cell'] = city_objects.apply(lambda row: findloc(row['lat'], 
                                           row['lon'],
                                          cell_center), axis=1)
    mapped_objects_dfs.append(city_objects)


100%|███████████████████████████████████████████| 16/16 [00:03<00:00,  5.25it/s]


In [13]:
all_mapped_objects_df = pd.concat(mapped_objects_dfs)

In [14]:
all_mapped_objects_df

,id,lat,lon,object_type,object_name,city,compound_obj,cell
0,87761966,55.694164,37.424646,highway,bus_stop,Москва,highwaybus_stop,"[125, 130]"
1,250559783,55.807910,37.418439,highway,bus_stop,Москва,highwaybus_stop,"[83, 129]"
2,252899807,55.731801,37.664628,highway,bus_stop,Москва,highwaybus_stop,"[111, 180]"
3,254393857,55.735781,37.656572,highway,bus_stop,Москва,highwaybus_stop,"[109, 178]"
4,254393882,55.729877,37.660870,highway,bus_stop,Москва,highwaybus_stop,"[112, 179]"
...,...,...,...,...,...,...,...,...
10,1932714919,48.802956,44.601331,amenity,courthouse,Волгоград,amenitycourthouse,"[34, 129]"
11,1972414614,48.755017,44.483780,amenity,courthouse,Волгоград,amenitycourthouse,"[51, 101]"
12,1972422818,48.690872,44.490803,amenity,courthouse,Волгоград,amenitycourthouse,"[75, 102]"
13,1972426150,48.656760,44.436448,amenity,courthouse,Волгоград,amenitycourthouse,"[88, 89]"


In [15]:
all_mapped_objects_df.to_csv("all_mapped_objects.csv", index=False)

In [16]:
all_objects[all_objects.id == 282413851]

,id,lat,lon,object_type,object_name,city,compound_obj
77,282413851,55.687808,37.573515,subway,yes,Москва,subwayyes
79,282413851,55.687808,37.573515,railway,station,Москва,railwaystation


In [17]:
all_mapped_objects_df['cell'] = all_mapped_objects_df['cell'].astype(str)

In [18]:
(all_mapped_objects_df[all_mapped_objects_df.object_name == 'bus_stop']
.groupby(['city', 'cell' ])['id'].count()).reset_index().sort_values(by='id')

,city,cell,id
0,Волгоград,"[100, 76]",1
9297,Москва,"[99, 216]",1
9298,Нижний Новгород,"[11, 10]",1
9304,Нижний Новгород,"[12, 10]",1
9307,Нижний Новгород,"[13, 15]",1
...,...,...,...
8194,Москва,"[76, 132]",12
5148,Москва,"[137, 142]",12
8488,Москва,"[82, 208]",13
7852,Москва,"[65, 141]",15
